# Azure AI Agent service - Many agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## Creaate connection to AI Foundry Project & instantiate a project client

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

## Instantiate AGENT 1 (Q&A)

In [5]:
agent1 = project_client.create_agent(
    model="gpt-4o",
    name="Q&A",
    instructions="You are an AI agent.",
    description="Q&A agent"
)

print(f"Created agent, agent ID: {agent1.id}")

Created agent, agent ID: asst_Sx8Jf8y4so0ig2waU3P8Jvu5


In [6]:
id1 = agent1.id

## Instantiate AGENT 2 (Translator)

In [7]:
agent2 = project_client.create_agent(
    model="gpt-4o-mini",
    name="Translator",
    instructions="You are an AI agent that translates English into Spanish.",
    description="Translator agent"
)

print(f"Created agent, agent ID: {agent2.id}")

Created agent, agent ID: asst_wd6p9YFkv2lLCZnk5DASlMd6


In [8]:
id2 = agent2.id

## Instantiate AGENT 3 (Emojis creator)

In [9]:
agent3 = project_client.create_agent(
    model="gpt-4o",
    name="Emojis",
    instructions="You are an AI agent that add emojis to a text",
    description="Emojis agent"
)

print(f"Created agent, agent ID: {agent3.id}")

Created agent, agent ID: asst_0daOF9qFp3Z6H2IRkyLAt0SC


In [10]:
id3 = agent3.id

### Show all agents

In [11]:
print(agent1)
print(agent2)
print(agent3)

{'id': 'asst_Sx8Jf8y4so0ig2waU3P8Jvu5', 'object': 'assistant', 'created_at': 1749546242, 'name': 'Q&A', 'description': 'Q&A agent', 'model': 'gpt-4o', 'instructions': 'You are an AI agent.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_wd6p9YFkv2lLCZnk5DASlMd6', 'object': 'assistant', 'created_at': 1749546245, 'name': 'Translator', 'description': 'Translator agent', 'model': 'gpt-4o-mini', 'instructions': 'You are an AI agent that translates English into Spanish.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_0daOF9qFp3Z6H2IRkyLAt0SC', 'object': 'assistant', 'created_at': 1749546246, 'name': 'Emojis', 'description': 'Emojis agent', 'model': 'gpt-4o', 'instructions': 'You are an AI agent that add emojis to a text', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}


### Create a thread for AGENT 1

In [12]:
thread = project_client.threads.create()
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_913NKgbaDtuZxvZLKYe5lJYW


### Create a message for AGENT 1

In [13]:
message = project_client.messages.create(thread_id=thread.id, role="user", content="Tell me about the 'Palacio de Bellas Artes' in Mexico City.")
print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_z4cPFgbwNm61hdzEjzQeDvGL


### Run AGENT 1

In [14]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent1.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_vArT55j5C887E6ye7aNRkcrT


### Check status of run for AGENT 1

In [15]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: completed


### Show usage (tokens)

In [16]:
run.usage

{'prompt_tokens': 74, 'completion_tokens': 661, 'total_tokens': 735, 'prompt_token_details': {'cached_tokens': 0}}

### Show me the stack of messages so far

In [17]:
messages = project_client.messages.list(thread_id=thread.id)

for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")

Message ID: msg_Xe2zb1staILgIUXJTnMbY8eT, Role: assistant, Content: [{'type': 'text', 'text': {'value': 'Certainly! The **Palacio de Bellas Artes** (Palace of Fine Arts) in **Mexico City** is one of the most iconic cultural landmarks in Mexico. It is renowned for its striking architecture and its role as a hub for arts and culture. Here’s an overview of its significance, history, and features:\n\n### **Historical Background**\n- Construction of the Palacio de Bellas Artes began in **1904** under the direction of the Italian architect **Adamo Boari**, who designed the building in a mixed style, combining Art Nouveau and Neoclassical elements. \n- The project was delayed by political unrest during the Mexican Revolution (1910–1920) and ongoing engineering challenges due to the soft, unstable soil beneath Mexico City.\n- Construction was completed in **1934** under the leadership of Mexican architect **Federico Mariscal**, who incorporated elements of **Art Deco** into the interior design

### Show last message of AGENT 1

In [18]:
messages = list(project_client.messages.list(thread_id=thread.id))

if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': 'Certainly! The **Palacio de Bellas Artes** (Palace of Fine Arts) in **Mexico City** is one of the most iconic cultural landmarks in Mexico. It is renowned for its striking architecture and its role as a hub for arts and culture. Here’s an overview of its significance, history, and features:\n\n### **Historical Background**\n- Construction of the Palacio de Bellas Artes began in **1904** under the direction of the Italian architect **Adamo Boari**, who designed the building in a mixed style, combining Art Nouveau and Neoclassical elements. \n- The project was delayed by political unrest during the Mexican Revolution (1910–1920) and ongoing engineering challenges due to the soft, unstable soil beneath Mexico City.\n- Construction was completed in **1934** under the leadership of Mexican architect **Federico Mariscal**, who incorporated elements of **Art Deco** into the interior design.\n- The Palace officially opened on **September 29, 1934**

### SHow bottom of stack of messages

In [19]:
if messages:
    last_message = messages[-1]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': "Tell me about the 'Palacio de Bellas Artes' in Mexico City.", 'annotations': []}}]


### Run AGENT 2

In [20]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent2.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_EyeZ8yub2X6l5Td0wDhbx2hN


### Check Status of the run for AGENT 2

In [21]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: completed


In [22]:
run.usage

{'prompt_tokens': 742, 'completion_tokens': 753, 'total_tokens': 1495, 'prompt_token_details': {'cached_tokens': 0}}

### SHow the stack of messages so far

In [23]:
messages = project_client.messages.list(thread_id=thread.id)

for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")

Message ID: msg_w1lWMmNkFQBiKhG5k4nxecfR, Role: assistant, Content: [{'type': 'text', 'text': {'value': '¡Claro! El **Palacio de Bellas Artes** en **Ciudad de México** es uno de los monumentos culturales más emblemáticos de México. Es conocido por su impresionante arquitectura y su papel como centro de artes y cultura. Aquí tienes un resumen sobre su significado, historia y características:\n\n### **Antecedentes Históricos**\n- La construcción del Palacio de Bellas Artes comenzó en **1904** bajo la dirección del arquitecto italiano **Adamo Boari**, quien diseñó el edificio en un estilo mixto, combinando elementos de Art Nouveau y Neoclásico.\n- El proyecto se retrasó debido a la agitación política durante la Revolución Mexicana (1910–1920) y los desafíos de ingeniería ocasionados por el suelo blando e inestable de la Ciudad de México.\n- La construcción se completó en **1934** bajo la dirección del arquitecto mexicano **Federico Mariscal**, quien incorporó elementos de **Art Deco** en 

### SHow last message of AGENT 2

In [24]:
messages = list(project_client.messages.list(thread_id=thread.id))

if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': '¡Claro! El **Palacio de Bellas Artes** en **Ciudad de México** es uno de los monumentos culturales más emblemáticos de México. Es conocido por su impresionante arquitectura y su papel como centro de artes y cultura. Aquí tienes un resumen sobre su significado, historia y características:\n\n### **Antecedentes Históricos**\n- La construcción del Palacio de Bellas Artes comenzó en **1904** bajo la dirección del arquitecto italiano **Adamo Boari**, quien diseñó el edificio en un estilo mixto, combinando elementos de Art Nouveau y Neoclásico.\n- El proyecto se retrasó debido a la agitación política durante la Revolución Mexicana (1910–1920) y los desafíos de ingeniería ocasionados por el suelo blando e inestable de la Ciudad de México.\n- La construcción se completó en **1934** bajo la dirección del arquitecto mexicano **Federico Mariscal**, quien incorporó elementos de **Art Deco** en el diseño interior.\n- El Palacio se inauguró oficialmente 

### Run AGENT 3

In [25]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent3.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_UvzWWxj905QygHNj2CcAcGNN


### Check status of run of AGENT 3

In [26]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: in_progress
Run status: completed


In [27]:
run.usage

{'prompt_tokens': 1498, 'completion_tokens': 140, 'total_tokens': 1638, 'prompt_token_details': {'cached_tokens': 0}}

### Show the Stack of messages so far

In [28]:
messages = project_client.messages.list(thread_id=thread.id)

for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")

Message ID: msg_eemFkEfC5GaMaiBmUuVpxGyd, Role: assistant, Content: [{'type': 'text', 'text': {'value': '🎭✨ ¡El **Palacio de Bellas Artes** en Ciudad de México es una joya cultural y arquitectónica! 🌟 Construido con mármol blanco de Carrara y coronado con una espectacular cúpula de colores vibrantes 🏛️🎨, es el hogar de conciertos, ópera, ballet y más. Su interior está decorado con **murales de grandes artistas mexicanos** como Diego Rivera y Siqueiros. 🇲🇽💃🎶 Además, es el escenario del famoso **Ballet Folklórico de México**. 💃🕺 Un imperdible a nivel turístico y artístico. 🖼️🎶', 'annotations': []}}]
Message ID: msg_w1lWMmNkFQBiKhG5k4nxecfR, Role: assistant, Content: [{'type': 'text', 'text': {'value': '¡Claro! El **Palacio de Bellas Artes** en **Ciudad de México** es uno de los monumentos culturales más emblemáticos de México. Es conocido por su impresionante arquitectura y su papel como centro de artes y cultura. Aquí tienes un resumen sobre su significado, historia y características:\n

### Print last message of AGENT 3

In [29]:
messages = list(project_client.messages.list(thread_id=thread.id))

last_message = messages[0]
print(f"Content: {last_message.content}")

Content: [{'type': 'text', 'text': {'value': '🎭✨ ¡El **Palacio de Bellas Artes** en Ciudad de México es una joya cultural y arquitectónica! 🌟 Construido con mármol blanco de Carrara y coronado con una espectacular cúpula de colores vibrantes 🏛️🎨, es el hogar de conciertos, ópera, ballet y más. Su interior está decorado con **murales de grandes artistas mexicanos** como Diego Rivera y Siqueiros. 🇲🇽💃🎶 Además, es el escenario del famoso **Ballet Folklórico de México**. 💃🕺 Un imperdible a nivel turístico y artístico. 🖼️🎶', 'annotations': []}}]


## Post processing

In [30]:
# List all agents in the project
print("Listing all agents in the project:")

agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_0daOF9qFp3Z6H2IRkyLAt0SC, Name: Emojis, Model: gpt-4o, Instructions: You are an AI agent that add emojis to a text
Agent ID: asst_wd6p9YFkv2lLCZnk5DASlMd6, Name: Translator, Model: gpt-4o-mini, Instructions: You are an AI agent that translates English into Spanish.
Agent ID: asst_Sx8Jf8y4so0ig2waU3P8Jvu5, Name: Q&A, Model: gpt-4o, Instructions: You are an AI agent.


In [31]:
project_client.delete_agent(id1)
print(f"Deleted agent, agent ID: {id1}")

Deleted agent, agent ID: asst_Sx8Jf8y4so0ig2waU3P8Jvu5


In [32]:
project_client.delete_agent(id2)
print(f"Deleted agent, agent ID: {id2}")

Deleted agent, agent ID: asst_wd6p9YFkv2lLCZnk5DASlMd6


In [33]:
project_client.delete_agent(id3)
print(f"Deleted agent, agent ID: {id3}")

Deleted agent, agent ID: asst_0daOF9qFp3Z6H2IRkyLAt0SC
